# Calculating the reserve Factors of the panels 

# Please enter your name 

In [1]:
name = "yannis"

# Imports 

In [2]:
import pandas as pd
import pandas as pd
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.columnbuckling import *
from formulas.panels import *
from formulas.strength import *
 

# Constants (Changed based on your data)

In [3]:
if name == 'yannis':
    sigma_yield = 490
    EModulus = 65241.07 
    nu = 0.34 
elif name == 'fabian':
    sigma_yield = 490
    EModulus = 65420.46
    nu = 0.34
elif name == 'daniel':
    sigma_yield = 490
    EModulus = 65669.47
    nu = 0.34
elif name == 'felix':
    sigma_yield = 490
    EModulus = 65669.47
    nu = 0.34

#Panel constants 
length = 750
width = 200


# Open stress values & properties of panels 

In [4]:
#Imoport panel stress file and reformat 
paneldf = pd.read_excel(f'../data/{name}/panel.xlsx', index_col=0)
paneldf = paneldf.drop(['sigmaZZ', 'sigmaXZ', 'sigmaYZ'],axis=1)
paneldf = paneldf.reset_index()
#Import panel properties and reformat
panelPropertiesdf = pd.read_excel(f'../data/{name}/panel_properties.xlsx', index_col=0)
panelPropertiesdf = panelPropertiesdf.reset_index()
panelPropertiesdf = panelPropertiesdf.rename(columns={'elements': 'Element ID'})

print(paneldf.head(5))
print(panelPropertiesdf.head(5))

   Element ID Component Name  sigmaXX  sigmaYY  sigmaXY        Load Case
0           1         panel1   -65.78   12.580    25.90  Subcase 1 (LC1)
1           2         panel1   -81.87   20.590    26.74  Subcase 1 (LC1)
2           3         panel1   -66.73   20.470    36.48  Subcase 1 (LC1)
3           4         panel2   -96.38    2.383    34.06  Subcase 1 (LC1)
4           5         panel2   -88.32   25.010    26.68  Subcase 1 (LC1)
   Element ID  thickness     mass
0           1          4  0.00054
1           2          4  0.00054
2           3          4  0.00054
3           4          4  0.00054
4           5          4  0.00054


# Split up the loadcases 

In [5]:
loadCase1df = paneldf[paneldf['Load Case'] == 'Subcase 1 (LC1)']
loadCase2df = paneldf[paneldf['Load Case'] == 'Subcase 2 (LC2)']
loadCase3df = paneldf[paneldf['Load Case'] == 'Subcase 3 (LC3)']
# Add the data to load case 
loadCase1df = pd.merge(loadCase1df, panelPropertiesdf, on='Element ID')
loadCase2df = pd.merge(loadCase2df, panelPropertiesdf, on='Element ID')
loadCase3df = pd.merge(loadCase3df, panelPropertiesdf, on='Element ID')

#Set index back to Element ID 
loadCase1df = loadCase1df.set_index('Element ID')
loadCase2df = loadCase2df.set_index('Element ID')
loadCase3df = loadCase3df.set_index('Element ID')

#Give some overview 
loadCase1df.head(5)

,Component Name,sigmaXX,sigmaYY,sigmaXY,Load Case,thickness,mass
Element ID,,,,,,,
1,panel1,-65.78,12.580,25.90,Subcase 1 (LC1),4,0.00054
2,panel1,-81.87,20.590,26.74,Subcase 1 (LC1),4,0.00054
3,panel1,-66.73,20.470,36.48,Subcase 1 (LC1),4,0.00054
4,panel2,-96.38,2.383,34.06,Subcase 1 (LC1),4,0.00054
5,panel2,-88.32,25.010,26.68,Subcase 1 (LC1),4,0.00054


# Now we need to combine into panels where we have columns:
-Panel ID (as our ID)
-sigmaXXavg
-sigmaYYavg
-sigmaXYavg
-length
-width
-thickness 


In [ ]:
# We can just average the stresses @fabianober Tackle rounding!!!! 
#For load case 1 
panelLC1 = loadCase1df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'median'
})
panelLC1['length'] = [length]*len(panelLC1)
panelLC1['width'] = [width]*len(panelLC1)
panelLC1

#For load case 2 
panelLC2 = loadCase2df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'median'
})
panelLC2['length'] = [length]*len(panelLC2)
panelLC2['width'] = [width]*len(panelLC2)
panelLC2

#For load case 3
panelLC3 = loadCase3df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'median'
})
panelLC3['length'] = [length]*len(panelLC3)
panelLC3['width'] = [width]*len(panelLC3)
panelLC3

panelLC1

,sigmaXX,sigmaYY,sigmaXY,thickness,length,width
Component Name,,,,,,
panel1,-71.460000,17.880000,29.706667,4.0,750,200
panel10,-71.460000,17.880000,29.706667,4.0,750,200
panel2,-93.236667,17.881000,29.606667,4.0,750,200
panel3,-93.370000,17.879333,29.613333,4.0,750,200
panel4,-92.716667,17.880000,30.066667,4.0,750,200
panel5,-92.223333,17.881000,52.546667,4.0,750,200
panel6,-92.223333,17.881000,52.546667,4.0,750,200
panel7,-92.716667,17.880000,30.066667,4.0,750,200
panel8,-93.370000,17.879333,29.613333,4.0,750,200


# Now we can apply the functions to the load cases 

In [8]:
panelLC1[['k_shear', 'k_biaxial', 'Reserve Factor']] = panelLC1.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC2[['k_shear', 'k_biaxial', 'Reserve Factor']] = panelLC2.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC3[['k_shear', 'k_biaxial', 'Reserve Factor']] = panelLC3.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC3

,sigmaXX,sigmaYY,sigmaXY,thickness,length,width,k_shear,k_biaxial,Reserve Factor
Component Name,,,,,,,,,
panel1,-23.740000,5.940667,118.433333,4.0,750,200,5.62,5.05,1.05
panel10,-23.740000,5.940667,118.433333,4.0,750,200,5.62,5.05,1.05
panel2,-30.973333,5.939667,118.066667,4.0,750,200,5.62,4.83,0.99
panel3,-31.016667,5.940667,118.066667,4.0,750,200,5.62,4.83,0.99
panel4,-30.804667,5.938000,119.830000,4.0,750,200,5.62,4.84,0.97
panel5,-30.636333,5.941000,209.466667,4.0,750,200,5.62,4.84,0.38
panel6,-30.636333,5.941000,209.466667,4.0,750,200,5.62,4.84,0.38
panel7,-30.804667,5.938000,119.830000,4.0,750,200,5.62,4.84,0.97
panel8,-31.016667,5.940667,118.066667,4.0,750,200,5.62,4.83,0.99


# Combining back into 1 Data Frame to output 